# Rendering Markdown and LaTeX in React

Recently I’ve been spending my nights and weekends learning React by building myself a new website. I got to the point of migrating my blog from Python (using [Pelican](https://blog.getpelican.com/)) to React. My blog posts are all written as Jupyter notebooks, then converted to HTML files and displayed.

Notebook files are really just giant JSON objects. This means I should be able to read a notebook directly into a JavaScript file and render it with a React component. Indeed, I created a Notebook component just for this purpose. I managed to render text and code (with syntax highlighting!) and images and it all looked quite nice. Example given:

![Blog post example](https://i.imgur.com/zNlmAZa.png)

I often include math expressions along with descriptive text. In notebooks, the expressions are written in LaTeX, in the middle of Markdown text, then rendered with MathJax. You’d type something like `$\sigma_U \sim \mathrm{Normal}(0, \Theta_U^2)$` and it would be rendered as

![Rendered expression example: sigma U from Normal distribution with parameters zero, theta U squared](https://i.imgur.com/5YFbxs5.png)

There is a not obvious conflict between Markdown and LaTeX in the line above. In LaTeX, underscores are used to denote subscripts. In Markdown, text between underscores is rendered as emphasis with `<em>` tags. LaTeX also makes heavy use of the backslash, it’s before every command such as `\sigma`. Backslashes are also used in Markdown for rendering special characters such as `\_` if you want to actually type an underscore instead of emphasizing some text. Conflicts abound. Markdown and MathJax fight against each other to render this text.

Given all this, my math expressions were not rendering correctly. When I was at Udacity, our engineering team tackled this same problem when trying to display math expressions in the classroom. I didn’t know all the details, but it was clear there was not a good solution. After some intense googling, I found [a GitHub issue](https://github.com/rexxars/react-markdown/issues/10#issuecomment-347763068) with most of the solution. It was a bit outdated however, so I had to make a few adjustments. Here is my solution to hopefully prevent future frustration. Let this be my contribution to society.

I’m using three packages to do this: `react-markdown`, `react-mathjax`, and `remark-math`. After installing those, this component will successfully (at least in my case) render math expressions written in LaTeX inside Markdown.

In [ ]:
import ReactMarkdown from 'react-markdown';
import MathJax from 'react-mathjax';
import RemarkMathPlugin from 'remark-math';

function MarkdownRender(props) {
    const newProps = {
        ...props,
        plugins: [
          RemarkMathPlugin,
        ],
        renderers: {
          ...props.renderers,
          math: (props) => 
            <MathJax.Node formula={props.value} />,
          inlineMath: (props) =>
            <MathJax.Node inline formula={props.value} />
        }
      };
      return (
        <MathJax.Provider input="tex">
            <ReactMarkdown {...newProps} />
        </MathJax.Provider>
      );
}

export default MarkdownRender

You use this just like a `ReactMarkdown` component,

```
...
render() {
  return <MarkdownRender source={cell.source.join("")} />
}
```

where `cell` is a Markdown cell from a notebook.

Using this I can now render beautiful math expressions written in a Jupyter notebook and displayed in a React app.

![Rendered math example](https://i.imgur.com/x5RRWm6.png)

If you’re running into similar problems, I hope this was helpful!